# P-Hacking

Das Konzept P-Hacking bedeutet, dass man absichtlich so lange die Auswertung der eigenen Daten verändert, bis genau die Werte herauskommen, die zur eigenen Hypothese passen.
Dieses Verhalten ist ein Problem für alle Bereiche:
In der Wissenschaft werden unter Umständen falsche Hypothesen (fälschlicherweise) empirisch untermauert.
In der Praxis bedeutet es, dass ggf. defekte Modelle in den Betrieb aufgenommen werden.
Sobald sich jemand auf das falsche Modell verlässt, kann dies zu ernsthaften Schäden an Menschen oder Umwelt führen.
Ebenso sind finanzielle Schäden nicht ausgeschlossen.
Weiterführende Infos gibt es z. B. auf
[Wikipedia](https://de.wikipedia.org/wiki/P-Hacking).

In [ ]:
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Zunächst wird wieder der Zufallsgenerator fixiert.

In [ ]:
np.random.seed(0)

In [ ]:
df = pd.DataFrame(np.random.randint(0, 100, size=(100, 26)), columns=list('ABCDEFGHIJKLMNOPQRSTUVWXYZ'))
df

In [ ]:
fig = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.xticks(range(df.shape[1]), df.columns)
plt.yticks(range(df.shape[1]), df.columns)
plt.colorbar()
plt.show()

## Feature-Engineering falsch gemacht

Nun werden wir so lange neue Features erstellen, bis wir (mindestens) eines finden, dass mit einem Attribut korreliert.
Hierhinter steht keinerlei Theorie, es sind reine Zufallsdaten.
Dafür werden die Attribute addiert und multipliziert.
Andere Operatoren wie Division und Subtraktion wären natürlich genauso möglich.
Darüber hinaus könnte man noch viel mehr Funktionen verwenden, wie Sinus, Logarithmus etc.

In [ ]:
# Hier werden die Additionen zweier Attribute gespeichert
additions = []

# Hier werden die Multiplikationen zweier Attribute gespeichert
multiplications = []

for column_A in df.columns:
    for column_B in df.columns:
        
        addition = df[column_A] + df[column_B]
        addition.name = f"{column_A} + {column_B}"
        additions.append(addition)
        
        multiplication = df[column_A] * df[column_B]
        multiplication.name = f"{column_A} * {column_B}"
        multiplications.append(multiplication)


Diese kreierten Attribute fügen wir nun dem DataFrame hinzu.

In [ ]:
df = df.assign(**{series.name : series for series in additions})
df = df.assign(**{series.name : series for series in multiplications})

df

In [ ]:
fig = plt.figure(figsize=(19, 15))
plt.matshow(df.corr(), fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.colorbar()
plt.show()

Es sind sehr viele Attribute geworden und dies lässt sich nun auch schwer visuell inspizieren.
Deswegen untersuchen wir diese Korrelationskoeffizienten nun weiter.
Dafür werden diese zunächst in eine Liste überführt.
Dadurch geht die Information verloren, wie genau die Zeilen heißen, die korrelieren.

In [ ]:
cor_values = df.corr().values.flatten()
cor_values = cor_values[cor_values != 1]  # Entferne die perfekten Korrelationen
pd.DataFrame(cor_values, columns=["Korrelationskoeffizient"]).plot.hist()
plt.show()

Sehr viele Korrelationen sind nur sehr schwach ausgeprägt.
Deswegen kann man diese im Histogramm besser erkennen als die stärkeren Korrelationen.
Um die stärker ausgeprägten Korrelationen besser sehen zu können, entfernen wir den mittleren Teil.
Außerdem ist weniger von Interesse, ob eine Korrelation positiv oder negativ ist.
Die hier eingesetzte Funktion `abs()` nimmt den absoluten Betrag.

In [ ]:
pd.DataFrame(abs(cor_values[((cor_values < -.5) | (cor_values > .5))])).plot.hist()
plt.show()

Es sieht so aus, als ob die Suche erfolgreich war.
Bei 0,9 gibt es mehrere Einträge.
Hier zoomen wir weiter heran.

In [ ]:
pd.DataFrame(abs(cor_values[((cor_values < -.9) | (cor_values > .9))])).plot.hist()
plt.show()

Nun isolieren wir die Korrelationen weiter durch Filtern.

In [ ]:
df_corr = df.corr()
np.fill_diagonal(df_corr.values, 0)
df_high_corr = df_corr[(df_corr > .9)]
df_high_corr

Das gleiche hier visualisiert:

In [ ]:
fig = plt.figure(figsize=(19, 15))
plt.matshow(df_high_corr, fignum=fig.number, cmap='RdBu', vmin=-1, vmax=1)
plt.colorbar()
plt.show()

In [ ]:
# remove all none rows here!
df_high_corr